In [79]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np

import io
import re
import string
import tqdm
from tensorflow.keras import layers

import seaborn as sns
from sklearn.metrics import pairwise

In [80]:
ds = tfds.load('multi_news', split='train', with_info=True)

#use index to get specific document
index = 20
count = 0
with tf.Graph().as_default():
    numpy_imgs = next(iter(ds))
    # numpy_imgs = tfds.as_numpy(ds)

In [81]:
count = 0
document = []
summary = []
for x in numpy_imgs:
    count += 1
    if count == index:
        # tf.print(x["document"])
        # print("\n")
        # print("\n")
        # print("SUMMARY")
        # tf.print(x["summary"])

        document = x["document"]
        summary = x["summary"]
        break

In [82]:
document = bytes(document.numpy())
document = [document.decode("utf-8")]

summary = bytes(summary.numpy())
summary = [summary.decode("utf-8")]

In [83]:
#create vocab
d_tokens = document[0].lower().split()
s_tokens = summary[0].lower().split()
tokens = d_tokens + s_tokens
vocab, index = {}, 1
vocab["<pad>"] = 0
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index = index + 1

inverse_vocab = {index: token for token, index in vocab.items()}
example = [vocab[word] for word in s_tokens]

In [84]:
embed = hub.load("https://tfhub.dev/google/nnlm-en-dim128/2")
embeddings = embed(["cat is on the mat", "dog is in the fog"])
print(embeddings)

tf.Tensor(
[[ 0.27107972 -0.01055073 -0.05728397  0.06853679 -0.08438271  0.22396211
  -0.00247001 -0.09797598 -0.06092518  0.01678422  0.0183306  -0.02683547
   0.01987647  0.02205245  0.0380337   0.02345292 -0.0535214  -0.02916854
  -0.13816142  0.255649    0.00548296  0.08994407  0.09702856 -0.01617393
   0.15273312  0.03449007  0.05599031  0.01964826 -0.01901525  0.11601479
   0.06575833 -0.03560898 -0.02412845 -0.00716866 -0.08950593 -0.01021391
   0.07431487 -0.10462939 -0.03951982  0.00272066 -0.01468687 -0.01350653
  -0.04825642  0.03088917 -0.0448269  -0.01743765  0.1034883   0.04149228
  -0.03979184  0.03878277  0.15273733 -0.09228262 -0.01723959  0.01830614
  -0.02075483  0.0800882  -0.08071491 -0.15573218  0.13893387  0.06140287
  -0.05639812 -0.05526257 -0.02765993 -0.175832    0.01034007 -0.19550695
   0.06209265 -0.03193678  0.08837268 -0.05629309  0.09268684  0.05395978
  -0.01900527 -0.17626993 -0.03103094 -0.12204378 -0.00157093 -0.07853678
  -0.05892268  0.11311311  

In [85]:
# stuff to do
# get vocabulary
# display output in a visual way
#lean word2vec

preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert = hub.load('https://tfhub.dev/google/experts/bert/wiki_books/2')

sentences = [
  "Here We Go Then, You And I is a 1999 album by Norwegian pop artist Morten Abel. It was Abel's second CD as a solo artist.",
  "The album went straight to number one on the Norwegian album chart, and sold to double platinum.",
  "Among the singles released from the album were the songs \"Be My Lover\" and \"Hard To Stay Awake\".",
  "Riccardo Zegna is an Italian jazz musician.",
  "Rajko Maksimović is a composer, writer, and music pedagogue.",
  "One of the most significant Serbian composers of our time, Maksimović has been and remains active in creating works for different ensembles.",
  "Ceylon spinach is a common name for several plants and may refer to: Basella alba Talinum fruticosum",
  "A solar eclipse occurs when the Moon passes between Earth and the Sun, thereby totally or partly obscuring the image of the Sun for a viewer on Earth.",
  "A partial solar eclipse occurs in the polar regions of the Earth when the center of the Moon's shadow misses the Earth.",
]

wordArray = []

for i in sentences:
  words = i.split()
  for w in words:
    w = w.replace(",", "")
    w = w.replace(".", "")
    wordArray.append(w)

print("word array")
print(wordArray)

bert_inputs = preprocess(sentences)
bert_outputs = bert(bert_inputs)
pooled_output = bert_outputs['pooled_output']
sequence_output = bert_outputs['sequence_output']

print('\nPooled output:')
print(pooled_output)
print('\nSequence output:')
print(sequence_output)


OSError: SavedModel file does not exist at: C:\Users\Brad\AppData\Local\Temp\tfhub_modules\602d30248ff7929470db09f7385fc895e9ceb4c0\{saved_model.pbtxt|saved_model.pb}

In [ ]:
def plot_similarity(features, labels):
  """Plot a similarity matrix of the embeddings."""
  cos_sim = pairwise.cosine_similarity(features)
  sns.set(font_scale=1.2)
  cbar_kws=dict(use_gridspec=False, location="left")
  g = sns.heatmap(
      cos_sim, xticklabels=labels, yticklabels=labels,
      vmin=0, vmax=1, cmap="Blues", cbar_kws=cbar_kws)
  g.tick_params(labelright=True, labelleft=False)
  g.set_yticklabels(labels, rotation=0)
  g.set_title("Semantic Textual Similarity")

plot_similarity(bert_outputs["pooled_output"], wordArray)

NameError: name 'bert_outputs' is not defined

In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
num_ns = 4

# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')

class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [86]:
def createWordVectors(textIdx, useBoard):
    file_path = textIdx
    text_ds = tf.data.TextLineDataset(file_path).filter(lambda x: tf.cast(tf.strings.length(x), bool))
    
    # Define the vocabulary size and number of words in a sequence.
    vocab_size = 4096
    sequence_length = 10

    # Use the TextVectorization layer to normalize, split, and map strings to
    # integers. Set output_sequence_length length to pad all samples to same length.
    vectorize_layer = layers.TextVectorization(
        standardize=custom_standardization,
        max_tokens=vocab_size,
        output_mode='int',
        output_sequence_length=sequence_length)

    vectorize_layer.adapt(text_ds.batch(1024))

    inverse_vocab = vectorize_layer.get_vocabulary()

    text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

    sequences = list(text_vector_ds.as_numpy_iterator())
    
    targets, contexts, labels = generate_training_data(
        sequences=sequences,
        window_size=2,
        num_ns=4,
        vocab_size=vocab_size,
        seed=SEED)

    targets = np.array(targets)
    contexts = np.array(contexts)[:,:,0]
    print("context")
    labels = np.array(labels)

    # BATCH_SIZE = 1024
    # BUFFER_SIZE = 10000
    BATCH_SIZE = 10
    BUFFER_SIZE = 15
    dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
    dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

    dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)

    embedding_dim = 128
    word2vec = Word2Vec(vocab_size, embedding_dim)
    word2vec.compile(optimizer='adam',
                    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])
    
    if useBoard:
        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")
        word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])
    else:
        word2vec.fit(dataset, epochs=20)

    weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
    vocab = vectorize_layer.get_vocabulary()
    return weights, vocab

In [87]:
# doc_weights, doc_vocab = createWordVectors("document.txt", False)
sum_weights, sum_vocab = createWordVectors("summary.txt", False)

print("document weights")
print(doc_weights)

print("summary weights")
print(sum_weights)

100%|██████████| 1/1 [00:00<00:00, 1004.38it/s]


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed